In [10]:
import torch
from transformers import CamembertForSequenceClassification, CamembertTokenizer, Trainer, TrainingArguments, EarlyStoppingCallback

import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from datasets import Dataset
import numpy as np

In [11]:
model = CamembertForSequenceClassification.from_pretrained("./best_model")
tokenizer = CamembertTokenizer.from_pretrained("./best_model")

In [12]:
def tokenize_function_test(exemples):
    email_encoding = tokenizer(exemples['email'], padding="max_length", truncation=True, max_length=64)
    return email_encoding

In [13]:
df_test = pd.read_csv('email_complet_test_copy.csv')

In [14]:
test_dataset_generer = Dataset.from_pandas(df_test)
test_dataset_generer = test_dataset_generer.map(tokenize_function_test, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [15]:
model.eval()

CamembertForSequenceClassification(
  (roberta): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0-11): 12 x CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias

In [16]:
all_preds = []


with torch.no_grad():
    for i in range(0, len(test_dataset_generer), 64):  # taille de batch de 64, ajustable
        batch = test_dataset_generer[i: i + 64]
        inputs = {k: torch.tensor(batch[k]) for k in batch.keys() if k in ["input_ids", "attention_mask"]}
        
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1).cpu().numpy()
        all_preds.extend(preds)


all_preds = np.array(all_preds)

print("Predictions:", all_preds)

# Si vous avez des étiquettes réelles pour évaluer, vous pouvez calculer la matrice de confusion et le rapport de classification
if 'label' in df_test.columns:
    from sklearn.metrics import confusion_matrix, classification_report
    labels = df_test['label'].values
    conf_matrix = confusion_matrix(labels, all_preds)
    class_report = classification_report(labels, all_preds)
    
    print("Confusion Matrix:\n", conf_matrix)
    print("Classification Report:\n", class_report)

Predictions: [1 0 0 ... 1 1 0]
Confusion Matrix:
 [[722 278]
 [ 50 950]]
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.72      0.81      1000
           1       0.77      0.95      0.85      1000

    accuracy                           0.84      2000
   macro avg       0.85      0.84      0.83      2000
weighted avg       0.85      0.84      0.83      2000



In [17]:
df_test['predicted_label'] = all_preds
incorrect_predictions = df_test[df_test['predicted_label'] != df_test['label']]
incorrect_predictions.to_csv('incorrect_predictions_test.csv', index=False)